### Preliminary Data Inspection

##### Getting data from Databricks catalog

In [0]:

df = spark.table("workspace.default.ab_nyc_2019")
display(df.limit(5))


id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,null,null,1,365
3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0


##### Basic info and shape

In [0]:
print(f"Total rows: {df.count()}")
print(f"Total columns: {len(df.columns)}")

Total rows: 49079
Total columns: 16


##### Exploring Table schema

In [0]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: long (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: long (nullable = true)
 |-- minimum_nights: long (nullable = true)
 |-- number_of_reviews: long (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: long (nullable = true)
 |-- availability_365: long (nullable = true)



##### Summary Statistics

In [0]:
display(df.describe())

summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,48895,49047,48735,48873,48894,48894,48891,48737,48894,48891,48893,48738,38863,48891,48737
mean,1.9017143236179568E7,1.02037532075E8,6.749845889059198E7,NaN,1.97400421E8,40.738701602564106,40.363071563477966,-73.9475341842947,154.36390484276728,152.22607432860855,7.119955821896795,23.257642906971974,1.3744471605383277,7.653883127774028,112.61000882286558
stddev,1.0983108385610096E7,8.709090084371348E7,7.855624635403281E7,NaN,null,0.06397153716374615,6.466708348941253,0.7354974291831097,513.660806114728,238.52839336353705,20.805063812845706,44.556023366233944,1.6943302852354718,34.82174958890119,131.6061838672227
min,2539,1 Bed Apt in Utopic Williamsburg,2438,'Cil,197400421,40.54268,-74.16254,-74.24442,-73.90783,0,0,0,0.0,0,0
max,36487245,"ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",274321313,현선,Woodside,Woodside,40.91306,40.68328,Shared room,10000,1250,629,58.5,365,365


##### Duplicates & Null summary per column

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import *
total_rows = df.count()
summary_rows = []
schema = dict(df.dtypes)
for column in df.columns:
  duplicate_df = df.groupBy(column).agg(count('*').alias('count')).filter(col('count')>1)
  duplicate_row_count = duplicate_df.agg({"count":"sum"}).collect()[0][0] 

  if duplicate_row_count is None:
      duplicate_row_count = 0

  duplicate_percentage = (duplicate_row_count / total_rows)*100 if total_rows > 0 else 0
  
  if schema[column] in ['int', 'bigint', 'double', 'float', 'decimal']:
      null_count = df.filter(col(column).isNull()|isnan(col(column))).count()
  else:
      null_count = df.filter(col(column).isNull()).count()

  null_percentage = (null_count / total_rows)*100 if total_rows > 0 else 0


  summary_rows.append(Row(
  column_name = column,
  duplicate_count = duplicate_row_count,
  duplicate_percentage = duplicate_percentage,
  null_count = null_count,
  null_percentage = null_percentage
  )) 

summary_df = spark.createDataFrame(summary_rows)
display(summary_df)

column_name,duplicate_count,duplicate_percentage,null_count,null_percentage
id,184,0.3749057641761242,184,0.3749057641761242
name,1699,3.4617657246480165,32,0.06520100246541291
host_id,16837,34.305914953442404,344,0.7009107765031888
host_name,42189,85.96140915666578,206,0.41973145337109563
neighbourhood_group,49045,99.93072393488049,185,0.37694329550316835
neighbourhood,48916,99.6678823936918,185,0.37694329550316835
latitude,41846,85.26253591148964,188,0.3830558894843008
longitude,43913,89.47411316448989,342,0.6968357138491004
room_type,49034,99.90831109028301,185,0.37694329550316835
price,48907,99.64954461174841,188,0.3830558894843008


##### Dropping nulls and assessing data loss in doing so

In [0]:
clean_df = df.dropna()

In [0]:
print(f"Original rows: {df.count()}")
print(f"Cleaned rows:{clean_df.count()}")

print(f"Rows removed: {df.count() - clean_df.count()}")
print(f"Percentage removed: {(df.count() - clean_df.count()) / df.count()*100}")

Original rows: 49079
Cleaned rows:38678
Rows removed: 10401
Percentage removed: 21.19236333258624


### Categorical Insights

#### room_type Insights

##### Flagging Invalid room_type values
Assuming ('Entire home/apt', 'Private room', 'Shared room') are valid room_type categories for insights. This field also consists of numeric and null values which do not give any information in relation to the room type in an AirBnB listing.

In [0]:
valid_room_type = ['Entire home/apt', 'Private room', 'Shared room']

df_room_flagged = df.select('id', 'room_type').withColumn('room_type_flagged', when(col('room_type').isin(valid_room_type), 'valid').otherwise('invalid'))
display(df_room_flagged)

id,room_type,room_type_flagged
2539,Private room,valid
2595,Entire home/apt,valid
3647,Private room,valid
3831,Entire home/apt,valid
5022,Entire home/apt,valid
5099,Entire home/apt,valid
5121,Private room,valid
5178,Private room,valid
5203,Private room,valid
5238,Entire home/apt,valid


##### room_type Popularity

In [0]:
valid_df = df.filter(col('room_type').isin(valid_room_type))
most_popular = valid_df.groupBy('room_type').count().orderBy(desc("count"))
least_popular = valid_df.groupBy('room_type').count().orderBy(asc("count"))
display(valid_df.groupBy('room_type').count().orderBy(desc("count")))

print("Most Popular room type:", most_popular.first()['room_type'])
print("Least Popular room type:", least_popular.first()['room_type'])

room_type,count
Entire home/apt,25348
Private room,22229
Shared room,1158


Most Popular room type: Entire home/apt
Least Popular room type: Shared room


##### Average Price per valid room_type



In [0]:
 most_expensive_room_type = valid_df.groupBy('room_type').agg(round(avg('price'),2).alias('avg_price')).orderBy(desc("avg_price"))
 least_expensive_room_type = valid_df.groupBy('room_type').agg(round(avg('price'),2).alias('avg_price')).orderBy(asc("avg_price"))
 display(price_per_room)
 print("Most Expensive Room Type: ", most_expensive_room_type.first()['room_type'])
 print("Least Expensive Room Type: ", least_expensive_room_type.first()['room_type'])

room_type,avg_price
Entire home/apt,211.88
Private room,89.51
Shared room,70.13


Most Expensive Room Type:  Entire home/apt
Least Expensive Room Type:  Shared room


#### Geographical Insights

##### Popularity of neighbourhood_group


In [0]:
neighbourhood_group_df = df.filter(col('neighbourhood_group').isNotNull()).groupBy('neighbourhood_group').agg(count('*').alias('count'))

display(neighbourhood_group_df)


neighbourhood_group,count
East Elmhurst,2
Greenwich Village,2
Crown Heights,3
Elmhurst,7
Woodhaven,3
Brooklyn,20057
SoHo,1
Bronx,1080
Bedford-Stuyvesant,9
Chelsea,2


##### Most Popular and Least Popular Neighbourhood_group

In [0]:

most_popular = neighbourhood_group_df.orderBy(desc("count")).limit(1)
least_popular = neighbourhood_group_df.orderBy(asc("count")).limit(1)

result = most_popular.union(least_popular)
display(result)

print("Most Popular Neighbourhood_group: ", most_popular.first()['neighbourhood_group'])
print("Least Popular Neighbourhood_group: ", least_popular.first()['neighbourhood_group'])

neighbourhood_group,count
Manhattan,21598
SoHo,1


Most Popular Neighbourhood_group:  Manhattan
Least Popular Neighbourhood_group:  SoHo


##### Price distribution across neighbourhood_group

In [0]:
price_dist_df = df.filter(col('neighbourhood_group').isNotNull()).groupBy("neighbourhood_group").agg(
    count("*").alias("listing_count"),
    avg("price").alias("avg_price"),
    min("price").alias("min_price"),
    max("price").alias("max_price"),
    stddev("price").alias("stddev_price")
).orderBy("avg_price", ascending=False)

display(price_dist_df)

neighbourhood_group,listing_count,avg_price,min_price,max_price,stddev_price
Manhattan,21598,196.72428002592832,0,10000,288.58852684878434
Brooklyn,20057,124.39771650795234,0,10000,187.03362186726287
Staten Island,370,114.99459459459459,13,5000,278.66814480677914
Queens,5630,99.57690941385435,10,10000,167.53040318660925
Bronx,1080,87.7287037037037,0,2500,107.19924849905865
Greenwich Village,2,55.5,31,80,34.64823227814083
Jackson Heights,2,16.0,2,30,19.79898987322333
Brooklyn Heights,1,14.0,14,14,null
Clinton Hill,1,14.0,14,14,null
Upper West Side,3,13.0,3,30,14.798648586948742


##### Most expensive and Least Expensive neighbourhood_group
Manhattan is the most expensive and East Elmhurst is the least expensive neighbourhood group

In [0]:
price_df = df.filter((col('neighbourhood_group').isNotNull()) & (col('price').isNotNull())).groupBy("neighbourhood_group").agg(
    avg("price").alias("avg_price"))

most_expensive = price_df.orderBy(desc("avg_price")).limit(1)
most_cheap = price_df.orderBy(asc("avg_price")).limit(1)
result = most_expensive.union(most_cheap)
display(result)

print("Most Expensive Neighbourhood_group: ", most_expensive.first()['neighbourhood_group'])
print("Most Economical Neighbourhood_group: ", most_cheap.first()['neighbourhood_group'])

neighbourhood_group,avg_price
Manhattan,196.72428002592832
East Elmhurst,1.0


Most Expensive Neighbourhood_group:  Manhattan
Most Economical Neighbourhood_group:  East Elmhurst


##### Popularity of Neighbourhood

In [0]:
display(df.filter(col('neighbourhood').isNotNull()).groupBy('neighbourhood').agg(count('*').alias('count')).orderBy(desc('count')))

neighbourhood,count
Williamsburg,3914
Bedford-Stuyvesant,3705
Harlem,2645
Bushwick,2460
Upper West Side,1968
Hell's Kitchen,1951
East Village,1849
Upper East Side,1791
Crown Heights,1561
Midtown,1541


##### Getting valid neighbourhood from the datasets since there are numeric values and nulls present in the field

In [0]:
valid_neighbourhood = df.filter(~col('neighbourhood').like("40%") & col("neighbourhood").isNotNull())

display(valid_neighbourhood.groupBy('neighbourhood').agg(count('*').alias('count')))


neighbourhood,count
Financial District,744
East Elmhurst,183
Greenwich Village,390
Dongan Hills,7
Crown Heights,1561
Morris Park,15
Carroll Gardens,233
Elmhurst,230
Willowbrook,1
Co-op City,2


##### Most Popular and Least Popular Neighbourhood
Williamsburg is the most popular and Willowbrook is the least popular neighbourhood

In [0]:
mp_neighbourhood = valid_neighbourhood.groupBy('neighbourhood').agg(count('*').alias('count')).orderBy(desc('count')).limit(1)
lp_neighbourhood = valid_neighbourhood.groupBy('neighbourhood').agg(count('*').alias('count')).orderBy(asc('count')).limit(1)
result = mp_neighbourhood.union(lp_neighbourhood)
display(result)

print("Most Popular Neighbourhood: ", mp_neighbourhood.first()['neighbourhood'])
print("Least Popular Neighbourhood: ", lp_neighbourhood.first()['neighbourhood'])

neighbourhood,count
Williamsburg,3914
Willowbrook,1


Most Popular Neighbourhood:  Williamsburg
Least Popular Neighbourhood:  Willowbrook


##### Most expensive and least expensive neighbourboods
Fort Wadsworth is the most expensive and Bull's Head is the least expensive neighbourhood

In [0]:
me_neighbourhood = valid_neighbourhood.filter(col('price').isNotNull()).groupBy('neighbourhood').agg(round(avg('price'),2).alias('avg_price')).orderBy(desc('avg_price')).limit(1)
le_neighbourhood = valid_neighbourhood.filter(col('price').isNotNull()).groupBy('neighbourhood').agg(round(avg('price'),2).alias('avg_price')).orderBy(asc('avg_price')).limit(1)

result = me_neighbourhood.union(le_neighbourhood)
display(result)

print("Most Expensive Neighbourhood: ", me_neighbourhood.first()['neighbourhood'])
print("Most Economical Neighbourhood: ", le_neighbourhood.first()['neighbourhood'])

neighbourhood,avg_price
Fort Wadsworth,800.0
Bull's Head,47.33


Most Expensive Neighbourhood:  Fort Wadsworth
Most Economical Neighbourhood:  Bull's Head


#### Host Analysis

##### Top 5 most busy hosts

In [0]:
display(df.filter((col('host_id').isNotNull()) & (col('host_name').isNotNull()) & (col('calculated_host_listings_count')>10)).select('host_id','host_name','calculated_host_listings_count').distinct().orderBy(desc('calculated_host_listings_count')).limit(5))


host_id,host_name,calculated_host_listings_count
219517861,Sonder (NYC),327
107434423,Blueground,232
30283594,Kara,121
137358866,Kazuya,103
12243051,Sonder,96


##### Most Expensive and Least Expensive Hosts


In [0]:
expensive_host = df.filter(col('host_id').isNotNull() & col('host_name').isNotNull() & col('price').isNotNull()).groupBy('host_name').agg(avg('price').alias('avg_price')).orderBy(desc('avg_price')).limit(1)
economical_host = df.filter(col('host_id').isNotNull() & col('host_name').isNotNull() & col('price').isNotNull()).groupBy('host_name').agg(avg('price').alias('avg_price')).orderBy(asc('avg_price')).limit(1)

result = expensive_host.union(economical_host)
display(result)

print("Most Expensive Host: ", expensive_host.first()['host_name'])
print("Most Economical Host: ", economical_host.first()['host_name'])

host_name,avg_price
Olson,9999.0
Qiuchi,0.0


Most Expensive Host:  Olson
Most Economical Host:  Qiuchi


#### Availability Analysis

##### Listings with high availability (greater than 200 days)

In [0]:
count_high_availability = df.filter(col('availability_365') > 200).count()
print(f"Listings with high availability: {count_high_availability}")


Listings with high availability: 13275


In [0]:


availability_by_room = valid_df.groupBy('room_type').agg(
    count('*').alias('listing_count'),
    avg('availability_365').alias('avg_availability'),
    max('availability_365').alias('max_availability'),
    min('availability_365').alias('min_availability'),
    stddev('availability_365').alias('stddev_availability')
).orderBy('avg_availability', ascending=False)

display(availability_by_room)


room_type,listing_count,avg_availability,max_availability,min_availability,stddev_availability
Shared room,1158,161.85837651122625,365,0,151.40196680384472
Entire home/apt,25348,111.84744358529272,365,0,129.8022904050754
Private room,22229,110.92401817445679,365,0,132.0546938773315
